In [3]:
import panel as pn
import param as pm
import pandas as pd
import numpy as np
from glob import glob
from bokeh.palettes import Category20, Turbo256
import hvplot.pandas
import millify
hvplot.extension('bokeh')
pn.extension()
# pd.set_option('display.max_rows', 5)

# Assuming other imports and setup are done as before


class Simulation(pm.Parameterized):
    experiment = pm.ObjectSelector(default='sanity_check_run', objects=sorted(list(set([f.split('/')[-1].split('-')[0] for f in glob('../data/simulations/*')]))))
    dataset = pm.Selector(default=None)  # This will be dynamically populated
    drop_list = pm.List(precedence=-1, default=['timestep', 'simulation', 'subset', 'timestep_in_days', 'block_time_in_seconds', 'delta_days', 'delta_blocks'])
    sim_df = pm.DataFrame(precedence=-1)
    max_rows = pm.Integer(20, bounds=(1, None), step=2)
    color_palette = pm.Selector(default=Category20, objects=[Category20, Turbo256], precedence=-1)
    column_colors = pm.Dict(precedence=-1)
    value_format = pm.Selector(default='Millify', objects=['Scientific', 'Millify', 'Decimal'])
    value_color = pm.Selector(default='Columns', objects=['Rows', 'Columns', 'None'])
    value_color_log_scale = pm.Boolean(False, precedence=-1)
    kpi_subset = pm.Selector()

    def __init__(self, **params):
        super(Simulation, self).__init__(**params)
        self._update_dataset_options()
        self._load_simulation_data()
        self._set_kpi_subsets()

    @pm.depends('experiment', watch=True)
    def _update_dataset_options(self, event=None):
        datasets = sorted(glob(f"../data/simulations/{self.experiment}*"))
        self.param.dataset.objects = datasets
        self.dataset = datasets[-1] if datasets else None

    @pm.depends('dataset', 'drop_list', watch=True)
    def _load_simulation_data(self):
        """Load the simulation data when dataset or drop_list are changed."""
        # Read pickle and drop uneccessary columns and reset index. Backfill data incase of nans in first block
        if self.dataset:
            self.sim_df = pd.read_pickle(self.dataset).drop(self.drop_list, axis=1).reset_index(drop=True).bfill()
            self._add_kpis()
        # If no dataset is set, initialize empty dataframe
        else:
            self.sim_df = pd.DataFrame()

    def _add_kpis(self):
        self.sim_df['issuance'] = self.sim_df['block_reward'] + self.sim_df['reference_subsidy']
        self.sim_df['fees'] = self.sim_df['compute_fee_volume'] + self.sim_df['storage_fee_volume']
        self._update_column_colors()

    def _set_kpi_subsets(self):
        all = 'All'
        fees_and_issuance = ['compute_fee_volume','storage_fee_volume', 'fees', 'block_reward', 'reference_subsidy', 'issuance']
        system_balances = ['other_issuance_balance', 'reward_issuance_balance']
        agent_balances = [
            'farmers_balance',
            'operators_balance',
            'nominators_balance',
            'holders_balance',
        ]
        agent_pool_balances = ['staking_pool_balance']
        protocol_treasury_balances = ['fund_balance']
        other_balances = list(set([c for c in self.sim_df.columns if 'balance' in c]) - set(system_balances + agent_balances + agent_pool_balances + protocol_treasury_balances) )
        supply_columns = list({c for c in self.sim_df.columns if 'supply' in c} - {'max_credit_supply', 'issued_supply', 'total_supply'})
        balance_columns = list(set([c for c in self.sim_df.columns if 'balance' in c]) - set(system_balances))
        
        KPI_SUBSETS = dict(
            all = all,
            fees_and_issuance = fees_and_issuance,
            system_balances = system_balances,
            agent_balances = agent_balances,
            agent_pool_balances = agent_pool_balances,
            protocol_treasury_balances = protocol_treasury_balances,
            other_balances = other_balances,
            supply_columns = supply_columns,
            balance_columns = balance_columns,
        )

        self.param.kpi_subset.objects = KPI_SUBSETS
        self.kpi_subset = all


    def _discrete_colorization(self):
        column_colors = {col: self.color_palette[20][i%20] for i, col in enumerate(self.sim_df.columns)}
        return column_colors

    def _continuous_colorization(self):
        column_colors = dict(zip(self.sim_df.columns, [self.color_palette[int(i)] for i in np.linspace(0,len(self.color_palette)-1, len(self.sim_df.columns))]))
        return column_colors
        
    @pm.depends('sim_df', 'color_palette', watch=True)
    def _update_column_colors(self):
        """Set column colors based on selected color palette and sim df"""
        if self.color_palette == Turbo256:
            self.column_colors = self._continuous_colorization()
        
        if self.color_palette == Category20:
            self.column_colors = self._discrete_colorization()

    def _truncate_dataframe(self, df):
        if self.max_rows >= len(self.sim_df):
            return self.sim_df
        else:
            return pd.concat([df.head(self.max_rows//2), df.tail(self.max_rows//2)])

    def kpi_subset_dataframe(self):
        return self.sim_df[self.kpi_subset]

    def styled_results_dataframe(self):
        def luminance(hex_color):
            # Convert hex color to RGB
            hex_color = hex_color.lstrip('#')
            r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
            # Calculate luminance
            return (0.299 * r + 0.587 * g + 0.114 * b) / 255

        def color_scale(val, min_val, max_val, log_scale=False):
            if pd.isnull(val) or (log_scale and val <= 0):
                return 'background-color: #ffffff; color: black'
            if log_scale:
                val, min_val, max_val = np.log(val), np.log(min_val), np.log(max_val)
            
            normalized = (val - min_val) / (max_val - min_val) if max_val > min_val else 0
            color_idx = int(normalized * (len(Turbo256) - 1))
            bg_color = Turbo256[color_idx]
            text_color = 'white' if luminance(bg_color) < 0.6 else 'black'
            return f'background-color: {bg_color}; color: {text_color}'

        def apply_color_styler(element, log_scale=False):
            for col in element.columns:
                min_val, max_val = element[col].min(), element[col].max()
                if log_scale:
                    # Adjust min_val to positive if necessary
                    min_val = min_val if min_val > 0 else 0.1
                element[col] = element[col].apply(color_scale, args=(min_val, max_val, log_scale))
            return element


        def format_value(x):
            """Format value based on the selected value_format, with type checking."""
            try:
                # Attempt to convert to float for formatting if possible.
                numeric_x = float(x)
                if self.value_format == 'Scientific':
                    return f"{numeric_x:.2e}"
                elif self.value_format == 'Decimal':
                    return f"{numeric_x:.f}"
                elif self.value_format == 'Millify':
                    return millify.millify(numeric_x, precision=2)
            except ValueError:
                # If conversion fails, return the original string.
                return x
            except TypeError:
                # If x is None or another type that cannot be converted to float.
                return x


        # Define the formatter to apply across all values in the DataFrame
        formatter = {col: format_value for col in self.sim_df.columns}

        header_styles = [{
            'selector': f'th.col_heading.level0.col{i}',
            'props': [('background-color', self.column_colors.get(col, '#ffffff')), ('color', 'black')]
        } for i, col in enumerate(self.sim_df.columns)]

        if self.kpi_subset == 'All':
            truncated_df = self._truncate_dataframe(self.sim_df)
        else:
            truncated_df = self._truncate_dataframe(self.kpi_subset_dataframe())

        # Apply color styler and value formatter
        styled_df = truncated_df.style.apply(apply_color_styler, log_scale=self.value_color_log_scale, axis=None).set_table_styles(header_styles).format(formatter)
        return styled_df

    def view_results_dataframe(self):
        styled_df = self.styled_results_dataframe()
        return pn.panel(styled_df, max_rows=self.max_rows)


    def runs_overview(self):
        return self.sim_df.groupby(['run', 'label', 'environmental_label']).size().reset_index(name='days').groupby(['label','environmental_label']).agg({'run': 'count', 'days': 'first'}).reset_index().rename({'run':'runs'},axis=1)

    def view_runs_overview(self):
        return pn.panel(self.runs_overview(), max_rows=self.max_rows)


    def view_color_columns_bar(self):
        """ View the colormap """

        # For some odd reason, hvplot reverses bar ordering when there are greater that 10 columns. So we apply a reverse to negate that hvplot bug. See here: https://github.com/holoviz/hvplot/issues/1277
        columns_reversed = self.sim_df.columns[::-1]
        
        return self.sim_df.count().to_frame().T.hvplot.bar(y=columns_reversed, color=[self.column_colors[c] for c in columns_reversed], rot=90, width=1400, height=500, title='Column Color Map', fontscale=1.4, yaxis=None)

    def view(self):
        """View the selected simulation results."""
        pd.set_option('display.max_rows', self.max_rows)
        view = pn.Column(
            """
            ## Simulation Analysis Dashboard
            """,
            pn.Accordion(
                ('Select Parameters', pn.Row(self, pn.Column('## Simulation Results DataFrame', self.view_results_dataframe))), 
                ('Runs Overview', self.view_runs_overview),
            ),
        )
        view[1].active = list(range(len(view[1])))
        # view[1].active = []
        # view[1].active = [0]
        return view


# Usage
s1 = Simulation()
df1 = s1.sim_df
s1.view()

Column
    [0] Markdown(str)
    [1] Accordion(active=[0, 1])
        [0] Row
            [0] Column(margin=(5, 10), name='Simulation')
                [0] StaticText(value='<b>Simulation</b>')
                [1] Select(name='Experiment', options=OrderedDict([('fund_inclus...]), value='sanity_check_run')
                [2] Select(name='Dataset', options=OrderedDict([('../data/sim...]), value='../data/simulations/sanit...)
                [3] IntInput(name='Max rows', start=1, step=2, value=20)
                [4] Select(name='Value format', options=OrderedDict([('Scientific'...]), value='Millify')
                [5] Select(name='Value color', options=OrderedDict([('Rows', ...]), value='Columns')
                [6] Select(name='Kpi subset', options=OrderedDict([('all', ...]), value='All')
            [1] Column
                [0] Markdown(str)
                [1] ParamMethod(method, _pane=DataFrame, defer_load=False)
        [1] ParamMethod(method, _pane=DataFrame, defer_load=False, name='Runs Overview')

In [4]:
s2 = Simulation(experiment='standard_stochastic_run')
df2 = s2.sim_df
s2.view()

Column
    [0] Markdown(str)
    [1] Accordion(active=[0, 1])
        [0] Row
            [0] Column(margin=(5, 10), name='Simulation')
                [0] StaticText(value='<b>Simulation</b>')
                [1] Select(name='Experiment', options=OrderedDict([('fund_inclus...]), value='standard_stochastic_run')
                [2] Select(name='Dataset', options=OrderedDict([('../data/sim...]), value='../data/simulations/stand...)
                [3] IntInput(name='Max rows', start=1, step=2, value=20)
                [4] Select(name='Value format', options=OrderedDict([('Scientific'...]), value='Millify')
                [5] Select(name='Value color', options=OrderedDict([('Rows', ...]), value='Columns')
                [6] Select(name='Kpi subset', options=OrderedDict([('all', ...]), value='All')
            [1] Column
                [0] Markdown(str)
                [1] ParamMethod(method, _pane=DataFrame, defer_load=False)
        [1] ParamMethod(method, _pane=DataFrame, defer_load=False, name='Runs Overview')